In [ ]:
pip install scikit-learn

In [ ]:
# 모듈 생성
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_1 = pd.read_csv("res8_dist3.csv")
df_2 = pd.read_csv("res8_coordinates.csv")

#df_1 = df_1.set_index(["Unnamed: 0"])
#df_2 = df_2.set_index(["Unnamed: 0"])

#df_1.drop("Unnamed: 0", axis=1, inplace=True)
#df_2.drop("Unnamed: 0", axis=1, inplace=True)

df = pd.merge(df_1, df_2, on="Unnamed: 0")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()


,유치원,초등학교,중학교,고등학교,대학교,디저트가게,탕후루가게,스타벅스,마라탕가게,음식점,...,병원,아파트,주점,학원,숙박업소,영화관,노래방,주차장,latitude,longitude
0,14.0,40.5,28.0,19.0,0.0,1455.0,11.0,16.0,42.5,4858.0,...,1381.5,158.0,1121.0,1357.5,6.0,1.0,365.5,242.0,37.551895,127.140409
1,10.0,28.5,17.5,20.0,0.0,527.0,2.5,10.0,24.0,1477.0,...,492.0,91.5,148.0,584.5,0.0,0.0,80.0,125.0,37.559936,127.167377
2,16.5,36.0,23.0,24.5,0.0,955.5,7.0,11.0,30.0,2908.5,...,895.5,137.5,565.5,1021.0,1.0,0.0,185.5,297.0,37.552761,127.156250
3,16.0,38.5,34.5,23.5,1.0,1606.5,11.0,14.5,40.5,5359.0,...,1510.0,172.0,1307.0,1439.0,6.0,2.0,420.5,193.5,37.538846,127.141929
4,15.5,33.0,25.5,14.5,1.0,1538.0,13.0,14.0,36.5,5148.0,...,1427.5,157.0,1219.0,1265.5,9.0,2.5,416.5,213.5,37.551462,127.132490


In [ ]:
"""
with open('/Users/min/Downloads/message.txt', 'r') as f:
    data = f.readlines()
data_split = [x.strip().split() for x in data]

df = pd.DataFrame(data_split)
df
"""

"\nwith open('/Users/min/Downloads/message.txt', 'r') as f:\n    data = f.readlines()\ndata_split = [x.strip().split() for x in data]\n\ndf = pd.DataFrame(data_split)\ndf\n"

In [ ]:
# 훈련세트와 테스트세트 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(df[['유치원', '초등학교', '중학교', '고등학교', '대학교', '디저트가게', '스타벅스', '마라탕가게', '음식점', '지하철역', '버스정거장', '병원', '아파트', '주점', '학원', '숙박업소', '영화관', '노래방', '주차장']],
                                                                      df["탕후루가게"], test_size=0.2, random_state=111)

grid_lat = df[['latitude']]
grid_lon = df[['longitude']]
tang = df[['탕후루가게']]

train_input1, test_input1, train_target1, test_target1 = train_test_split(train_input, train_target, test_size=0.2, random_state=111)




In [ ]:
test_target

243      2.0
727      3.0
271      7.5
1154     9.5
1155     5.5
        ... 
763     14.5
1142     0.0
414     10.0
1082     7.0
741      4.5
Name: 탕후루가게, Length: 238, dtype: float64

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input1)
train_scaled = ss.transform(train_input1)
test_scaled = ss.transform(test_input1)
test2_scaled = ss.transform(test_input)

In [ ]:
# PolynomialFeatures 변환기 사용
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False)
poly.fit(train_scaled)	                    # 변환을 하기 위해서는 훈련이 먼저 필요하다!
train_poly = poly.transform(train_scaled)	# 훈련 세트 변환
test_poly = poly.transform(test_scaled)		# 테스트 세트 변환
test_poly2 = poly.transform(test2_scaled)

#poly2 = PolynomialFeatures(include_bias=False)
#poly2.fit(test_input)

In [ ]:
from sklearn.linear_model import LinearRegression

# k-최근접 이웃 회귀 알고리즘
lr = LinearRegression()

# 모델 훈련
lr.fit(train_poly, train_target1)

# 결정계수 구하기
print("train score:", lr.score(train_poly, train_target1))
print("test score:", lr.score(test_poly, test_target1))

train score: 0.9554379560855426
test score: 0.8871205925678077


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

scores = cross_validate(lr, test_input, test_target)
print(np.mean(scores['test_score']))


lat_lon_df = df.iloc[test_input.index][:]
lat_lon_df = lat_lon_df[['latitude', 'longitude']]


0.7789144850370465


In [ ]:
predict_li = lr.predict(test_poly2).tolist()

predict_li = [round(i, 5) for i in predict_li]



In [ ]:
lat_lon_df['예측값'] = predict_li
lat_lon_df['실제값'] = test_target
lat_lon_df['MSE'] = (lat_lon_df['예측값'] - lat_lon_df['실제값'])**2
print(lat_lon_df)

# sum = 0
"""
for i in range(len(lat_lon_df)):
  a = lat_lon_df.iloc[i][lat_lon_df.columns.get_loc('예측값')]
  b = lat_lon_df.iloc[i][lat_lon_df.columns.get_loc('실제값')]

  sum += (a-b)**2
"""
# print(sum/len(lat_lon_df))

# lat_lon_df.to_csv('/content/lat_lon_df.csv')



       latitude   longitude       예측값   실제값        MSE
243   37.504930  127.002691   0.54035   2.0   2.130578
727   37.612770  126.938208   2.92697   3.0   0.005333
271   37.493210  127.027914   5.29251   7.5   4.873012
1154  37.577262  127.006050  14.23704   9.5  22.439548
1155  37.582220  126.977626   6.24198   5.5   0.550534
...         ...         ...       ...   ...        ...
763   37.640592  127.082202  13.96757  14.5   0.283482
1142  37.620858  126.965076  -0.73296   0.0   0.537230
414   37.539133  126.911271   9.43365  10.0   0.320752
1082  37.536400  126.979096   6.61875   7.0   0.145352
741   37.611868  126.922419   7.65106   4.5   9.929179

[238 rows x 5 columns]


"\nfor i in range(len(lat_lon_df)):\n  a = lat_lon_df.iloc[i][lat_lon_df.columns.get_loc('예측값')]\n  b = lat_lon_df.iloc[i][lat_lon_df.columns.get_loc('실제값')]\n\n  sum += (a-b)**2\n"

In [ ]:
lat_lon_df.describe()

,latitude,longitude,예측값
count,238.000000,238.000000,238.000000
mean,37.551049,126.998663,7.606808
std,0.058249,0.092661,5.214508
min,37.442374,126.771135,-1.397166
25%,37.505141,126.929038,3.183096
50%,37.547426,127.009090,7.146455
75%,37.594671,127.073396,11.291378
max,37.695917,127.180022,22.499431


In [ ]:
df0 = lat_lon_df.loc[lat_lon_df['실제값'] == 0]

df0.describe()

,latitude,longitude,예측값,실제값,MSE
count,23.000000,23.000000,23.000000,23.0,23.000000
mean,37.510152,126.974496,0.773520,0.0,2.123668
std,0.074082,0.097863,1.262802,0.0,3.346524
min,37.442374,126.771135,-1.397166,0.0,0.000388
25%,37.462786,126.955425,0.049461,0.0,0.083618
50%,37.473009,127.007316,0.329936,0.0,0.340583
75%,37.550973,127.034936,1.702699,0.0,2.924091
max,37.695917,127.109979,3.515978,0.0,12.362102


In [ ]:
"""
# 다중 회귀 모델 훈련하기 = 선형 회귀 모델을 훈련하는 것과 같음
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_poly, train_target)
print(lr.score(train_poly, train_target))	# 0.9903183436982125
print(lr.score(test_poly, test_target))
"""

'\n# 다중 회귀 모델 훈련하기 = 선형 회귀 모델을 훈련하는 것과 같음\nfrom sklearn.linear_model import LinearRegression\n\nlr = LinearRegression()\nlr.fit(train_poly, train_target)\nprint(lr.score(train_poly, train_target))\t# 0.9903183436982125\nprint(lr.score(test_poly, test_target))\n'

In [ ]:
"""
# Plotting linear
plt.plot(x, lin.forward(x), label='Prediction')

# Plotting test data points
plt.scatter([i for idx, i in enumerate(X_test)],
            [i for idx, i in enumerate(y_test)],
            label='Test', marker='x', color='r')

plt.title(f'Data points (MSE: {mse:0.3f})')
plt.xlabel('x')
plt.ylabel('y')
plt.ylim([-20, 20])
plt.legend()
plt.show()
"""